In [ ]:
#load all the datasets
cs_positives =  pd.read_csv('/content/drive/My Drive/RefD-dataset-master/Course/CS.edges', sep='\t', header=None)
cs_negatives =  pd.read_csv('/content/drive/My Drive/RefD-dataset-master/Course/CS.edges_neg', sep='\t', header=None)
math_positives =  pd.read_csv('/content/drive/My Drive/RefD-dataset-master/Course/MATH.edges', sep='\t', header=None)
math_negatives =  pd.read_csv('/content/drive/My Drive/RefD-dataset-master/Course/MATH.edges_neg', sep='\t', header=None)
#unite the positive datasets
#the structure is A,B where B is prereq of A
positives = pd.concat([cs_positives,math_positives])
#we obtain the negative counterpart of the positive dataset because:
#if B is prereq of A, then A is not prereq of B
cols = list(positives.columns)
cols = cols[1::2] + cols[::2]
neg = positives[cols]
#now we concat it to the other negative datasets
negatives = pd.concat([neg,cs_negatives,math_negatives])
#now we add a column with only 1s to the positive dataset and a column of only 0s to the negative one
#this way the dataset follows the same standard as the Prelearn one
positives[2] = 1
negatives[2] = 0

#We create the train and validation dataset like this because we have very few positives compared to the negatives
#so we spread them more fairly
positives_train = positives.sample(frac=0.8,random_state=200) 
positives_validation = positives.drop(positives_train.index)
negatives_train = negatives.sample(frac=0.8,random_state=200) 
negatives_validation = negatives.drop(negatives_train.index)
train = pd.concat([positives_train,negatives_train])
validation = pd.concat([positives_validation,negatives_validation])
#we shuffle the dataframe to create randomness
train = train.sample(frac=1)
validation = validation.sample(frac=1)

#df is needed for later, it's the whole dataset
df = pd.concat([positives,negatives])
#we shuffle the dataframe since right now it would give trouble while creating a proper training and validation set
df = df.sample(frac=1)


In [ ]:
wiki_wiki = wikipediaapi.Wikipedia('en',extract_format=wikipediaapi.ExtractFormat.WIKI)

#create dictionary of tokenized documents
punct = string.punctuation + '«``»' + "''"
en_stop_words = nltk.corpus.stopwords.words('english')

doc_dict = {}

#we get the english wikipedia pages

for index,row in df.iterrows():
  #Titles of the two documents
  title_A = row[0]
  title_B = row[1]
  
  #we check if the page related to the titles is already in the dictionary
  if title_A not in doc_dict:
    p_wiki = wiki_wiki.page(title_A)
    document = (p_wiki.title + " " + p_wiki.text).replace("'"," ")
    #document to lowercase, replaced apostrophe with space since the tokenizer isn't able to split words like "l'addizione"
    document = document.replace("\\"," ")
    document = document.replace("displaystyle"," ")
    #tokenized
    tokenized = nltk.tokenize.word_tokenize(document, "english")
    #no punctuation and lowercase
    no_punct = [x.lower() for x in tokenized if x not in punct]
    #remove stop words
    no_stop_words = [x for x in no_punct if x not in en_stop_words]
    #add tokenized document
    doc_dict[title_A] = no_stop_words

  if title_B not in doc_dict:
    p_wiki = wiki_wiki.page(title_B)
    document = (p_wiki.title + " " + p_wiki.text).replace("'"," ")
    #document to lowercase, replaced apostrophe with space since the tokenizer isn't able to split words like "l'addizione"
    document = document.replace("\\"," ")
    document = document.replace("displaystyle"," ")
    #tokenized
    tokenized = nltk.tokenize.word_tokenize(document, "english")
    #no punctuation and lowercase
    no_punct = [x.lower() for x in tokenized if x not in punct]
    #remove stop words
    no_stop_words = [x for x in no_punct if x not in en_stop_words]
    #add tokenized document
    doc_dict[title_B] = no_stop_words
 